<a href="https://colab.research.google.com/github/profteachkids/CHE5136_Fall2021/blob/main/Secant_Newton_inclass.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
from plotly.subplots import make_subplots
import plotly.io as pio
pio.templates.default='plotly_dark'
eps= np.finfo(np.float64).eps

In [ ]:
def f(x):
    return (x-3.)*(x+1.)*(x+3.)*(x-6.)

In [ ]:
x=np.linspace(-4,7,100)
fig=make_subplots()
fig.add_scatter(x=x,y=f(x), mode='lines')
fig.update_layout(width=800,height=600)

In [ ]:


x1, x2 = 2., 7. 
xlist = []
xlist.append(x1)
xlist.append(x2)
for i in range(100):
    y1,y2=f(x1),f(x2)
    m = (y2-y1)/(x2-x1 + eps)
    x3 = x2 - y2/(m + eps)
    xlist.append(x3)
    if np.abs(x2-x1) < 1e-10:
        break
    x1= x2
    x2=x3

xlist = np.array(xlist)
ylist = f(xlist)
labels = list(range(1, xlist.size+1))
fig2=make_subplots()
fig2.add_scatter(x=x,y=f(x), mode='lines')
fig2.add_scatter(x=xlist,y=ylist, mode='markers' , customdata= labels, hovertemplate='%{customdata}')
fig2.update_layout(width=800,height=600)

In [ ]:
f(xlist)

TypeError: ignored